# Cómo crear embeddings con langchain

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")



# Qdrant

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [4]:
import os

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

In [19]:

client = QdrantClient(
    url=url,
    api_key=api_key,
    https=True,
    timeout=300
)
client.create_collection(
    collection_name="book-collection-3",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

True

In [20]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="book-collection-3",
    embedding=embeddings,
)

# Load document

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/fortaleza-digital.pdf")
pages = loader.load()


In [8]:
filtered_documents = [page for page in pages if len(page.page_content) > 0]

In [12]:
import uuid
ids = [str(uuid.uuid4()) for _ in range(len(filtered_documents))]

# Ingest Documents

In [13]:
import uuid
from langchain_core.documents import Document

In [21]:

vector_store.add_documents(documents=filtered_documents[:100], ids=ids)

['43232f04-2796-4e96-88b7-3814b68af282',
 'e3b0e6b2-a102-436d-8c29-c07bdcbad97d',
 'ed7c9ea3-b963-435f-90f9-6f0d805c2a8e',
 'cc7e03f2-d602-4654-9e58-ee1c2cf29beb',
 '9cfb67bf-e4ff-4d63-b4fc-8ce04d75de55',
 '7f89cb61-cebd-405e-ba4d-03e85ecbd917',
 '33998ac6-beb5-4100-a32f-f8add1c439d4',
 '976a6d18-c9f4-42ab-b4b4-c09872c62ec0',
 '1d941b1e-bd23-4fcf-8deb-2d9c5d2e1ff6',
 '464a6fcf-eeda-42ab-8b6e-c4854eaee9b8',
 '6d48e848-e4e8-406b-8f6d-27817c3ca1e0',
 '74dcb98b-7e2d-40bd-88d9-9dda7656be66',
 '491adb7e-457f-4304-9a1b-5e8f4ce3eeca',
 '2fb15d14-1ccf-4cc9-9981-d79d48dcd36f',
 '7e2e21a9-3b56-46fc-b301-61cd19ceb9a4',
 '71335f72-f2d1-4578-a146-1658ae5fb1fa',
 '1c38c87e-d33a-40fa-8788-843d36b4927f',
 '241260ee-e909-4c4f-a70a-2b0715b64a1c',
 '4c1ea896-34c0-482a-be1c-b6b86b7a0c5c',
 '4b2c7f25-a82b-4fd4-8b6e-809d14b96f8a',
 '59a9f7a6-27be-42a1-a81f-a77c1b18b4ba',
 '0a23683d-84b3-475e-ad18-f4c592881998',
 '3002e2bb-8856-4eef-9b9c-399ac7e724d2',
 'f03caf60-25af-4610-80ea-4c9a2c807241',
 '6011306d-4882-

# Make a search

In [22]:
vector_store.similarity_search(query="quién es susan?", k=4)

[Document(metadata={'source': 'data/fortaleza-digital.pdf', 'page': 2, 'page_label': '3', '_id': 'e3b0e6b2-a102-436d-8c29-c07bdcbad97d', '_collection_name': 'book-collection-3'}, page_content='Susan Fletcher, la criptógrafa estrella de la ultrasecreta Agencia de\nSeguridad Nacional (NSA) no puede dar crédito a sus oídos cuando su jefe,\nel subdirector de la Agencia, le informa de que han interceptado un código\nque ni siquiera la mayor supercomputadora conocida puede descifrar. La\núnica pista para romper el letal código parece estar oculta en el cadáver de\nun hombre que ha fallecido en España, donde ha sido enviado David, el\nprometido de Susan. Mientras éste intenta hallar la clave y sobrevivir a la\npersecución de un metódico e implacable asesino a sueldo en las calles de\nSevilla, Susan se enfrentará a su propio drama en las instalaciones de\nmáxima seguridad de la NSA, durante una larga noche en la que la mentira\ny el asesinato acechan tras cada puerta.'),
 Document(metadata={'s

# Use filters

In [ ]:
from qdrant_client import models

results = vector_store.similarity_search(
    query="programming",
    k=1,
    filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.theme",
                match=models.MatchValue(
                    value="software"
                ),
            ),
        ]
    ),
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")